In [ ]:
import sys, os; sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__) if '__file__' in globals() else os.getcwd(), '..')))
#import os; os.chdir(os.path.dirname(os.getcwd()))
from utils.model_loader import get_model_fits
import numpy as np
import pandas as pd
import re
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
data_dir = f"datasets/friedman"
#results_dir_relu = "results/regression/single_layer/relu/friedman/convergence"
results_dir_tanh = "results/regression/single_layer/tanh/friedman/convergence_extensive"
#model_names_relu = ["Gaussian", "Regularized Horseshoe", "Dirichlet Horseshoe", "Dirichlet Student T"]
#model_names_tanh = ["Gaussian tanh", "Regularized Horseshoe tanh", "Dirichlet Horseshoe tanh nodewise", "Dirichlet Student T tanh nodewise"]
model_names_tanh = ["Dirichlet Horseshoe tanh nodewise"]

#relu_fits = {}
tanh_fits = {}

files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    # relu_fit = get_model_fits(
    #     config=full_config_path,
    #     results_dir=results_dir_relu,
    #     models=model_names_relu,
    #     include_prior=False,
    # )
    
    tanh_fit = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh,
        models=model_names_tanh,
        include_prior=False,
    )
    

    #relu_fits[base_config_name] = relu_fit  # use clean key
    tanh_fits[base_config_name] = tanh_fit  # use clean key
    


In [ ]:
data_dir = f"datasets/friedman/many"
model_names_tanh = ["Dirichlet Horseshoe tanh nodewise"]

files = sorted(f for f in os.listdir(data_dir) if f.endswith(".npz"))
for fname in files:
    base_config_name = fname.replace(".npz", "")  # e.g., "GAM_N100_p8_sigma1.00_seed1"
    full_config_path = f"{base_config_name}"  # → "type_1/GAM_N100_p8_sigma1.00_seed1"
    # relu_fit = get_model_fits(
    #     config=full_config_path,
    #     results_dir=results_dir_relu,
    #     models=model_names_relu,
    #     include_prior=False,
    # )
    
    tanh_fit = get_model_fits(
        config=full_config_path,
        results_dir=results_dir_tanh,
        models=model_names_tanh,
        include_prior=False,
    )
    

    #relu_fits[base_config_name] = relu_fit  # use clean key
    tanh_fits[base_config_name] = tanh_fit  # use clean key
    


In [3]:
import pandas as pd
import numpy as np
import arviz as az

seeds = [2] #[1, 2, 11, 16]

def get_N_sigma(seed):
    if seed == 1:
        N=100
    elif seed == 2:
        N=200
    elif seed == 16:
        N=50
    else:
        N=500
    sigma = 1
    return N, sigma

def get_all_convergence_diagnostics(all_fits):
    diagnostics = []
    rhats_global = {}
    for config_name, model_fits in all_fits.items():
        rhats = {}
        for model_name, fit in model_fits.items():
            try:
                idata = az.from_cmdstanpy(fit['posterior'])
                y_pred = fit['posterior'].stan_variable('output_test')
                
                path = f'datasets/friedman/{config_name}.npz'
                try:
                    data = np.load(path)
                    y_test = data["y_test"]
                except FileNotFoundError:
                    print(f"[SKIP] File not found: {path}")
                    continue
                
                divergent = idata.sample_stats["diverging"].values  # shape: (n_chains, n_draws)
                divergent_flat = divergent.flatten()  # shape: (8000,)
                divergences = np.sum(divergent_flat)
                y_pred_no_div = y_pred[~divergent_flat]
                S = y_pred.shape[0]
                rmses = np.zeros(S)
                rmses_no_div = np.zeros(S - np.sum(divergent_flat))
                
                for i in range(S):
                   rmses[i] = np.sqrt(np.mean((y_pred[i].squeeze() - y_test.squeeze()) ** 2))
                
                for i in range(S - np.sum(divergent_flat)):
                    rmses_no_div[i] = np.sqrt(np.mean((y_pred_no_div[i].squeeze() - y_test.squeeze()) ** 2))

                summary = az.summary(idata, var_names=["output"], round_to=3)
                
                rhat = summary["r_hat"]
                
                rhats[model_name] = rhat

                ess_bulk = summary["ess_bulk"]
                ess_tail = summary["ess_tail"]
                ess = summary["ess_tail"]
                
                try:
                    seed = int(config_name.split("_seed")[-1])
                    N, sigma = get_N_sigma(seed)
                except:
                    N, sigma = np.nan, np.nan

                diagnostics.append({
                    "model": model_name,
                    "max_rhat": rhat.max(),
                    "median_rhat": rhat.median(),
                    "prop_divergent": divergences/S,
                    #"rmse": np.mean(rmses, axis=0),
                    #"rmse_no_div": np.mean(rmses_no_div, axis=0),
                    "median_ess_tail": ess_tail.median()/S,
                    "median_ess_bulk": ess_bulk.median()/S,
                    "N": N,
                    #"sigma": sigma,
                })

            except Exception as e:
                diagnostics.append({
                    #"config": config_name,
                    "model": model_name,
                    "max_rhat": np.nan,
                    "median_rhat": np.nan,
                    "p95_rhat": np.nan,
                    "min_ess_bulk": np.nan,
                    "min_ess_tail": np.nan,
                    #"n_divergent": np.nan,
                    "error": str(e)
                })
        rhats_global[config_name] = rhats

    return pd.DataFrame(diagnostics), rhats_global


In [4]:
#relu_diagostic, rhats_relu = get_all_convergence_diagnostics(relu_fits)
tanh_diagostic, rhats_tanh = get_all_convergence_diagnostics(tanh_fits)

In [5]:
# relu_grouped = relu_diagostic.assign(row_index=lambda df: df.index) \
#     .sort_values(["model", "row_index"]) \
#     .drop(columns="row_index") \
#     .reset_index(drop=True)
tanh_grouped = tanh_diagostic.assign(row_index=lambda df: df.index) \
    .sort_values(["model", "row_index"]) \
    .drop(columns="row_index") \
    .reset_index(drop=True)


In [ ]:
#print(relu_grouped.to_latex(index=False))
print((tanh_grouped.round(3)).to_latex(index=False))

## CONVERGENCE vs ERROR

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import arviz as az

fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharex=True, sharey=True)

# --- First panel: N=100 ---
dataset_path = "datasets/friedman/Friedman_N100_p10_sigma1.00_seed1.npz"
data = np.load(dataset_path)
y_test = data["y_test"]
models_dict = tanh_fits['Friedman_N100_p10_sigma1.00_seed1']

for model_name, model_dict in models_dict.items():
    model = model_dict['posterior']
    idata = az.from_cmdstanpy(model)
    rhat = az.summary(idata, var_names=["output_test"], round_to=3)["r_hat"].values
    y_pred = np.mean(model.stan_variable("output_test"), axis=0).squeeze(-1)
    rmse = np.sqrt((y_test - y_pred) ** 2)
    axes[0].scatter(rhat, rmse, label=model_name, alpha=0.7)

axes[0].set_title("N = 100")
axes[0].set_xlabel(r"$\hat{R}$")
axes[0].set_ylabel("RMSE per test point")
axes[0].grid(True)

# --- Second panel: N=200 ---
dataset_path = "datasets/friedman/Friedman_N200_p10_sigma1.00_seed2.npz"
data = np.load(dataset_path)
y_test = data["y_test"]
models_dict = tanh_fits['Friedman_N200_p10_sigma1.00_seed2']

for model_name, model_dict in models_dict.items():
    model = model_dict['posterior']
    idata = az.from_cmdstanpy(model)
    rhat = az.summary(idata, var_names=["output_test"], round_to=3)["r_hat"].values
    y_pred = np.mean(model.stan_variable("output_test"), axis=0).squeeze(-1)
    rmse = np.sqrt((y_test - y_pred) ** 2)
    axes[1].scatter(rhat, rmse, label=model_name, alpha=0.7)

axes[1].set_title("N = 200")
axes[1].set_xlabel(r"$\hat{R}$")
axes[1].grid(True)

# Shared legend
handles, labels = axes[1].get_legend_handles_labels()
fig.legend(handles, labels, loc="upper center", ncol=len(labels))

plt.tight_layout()
plt.subplots_adjust(top=0.88)
plt.show()


## TRACEPLOTS

In [ ]:
import arviz as az
gauss_fit = tanh_fits['Friedman_N200_p10_sigma1.00_seed2']['Gaussian tanh']['posterior']
idata = az.from_cmdstanpy(gauss_fit)
divergent = idata.sample_stats["diverging"].values  # shape (n_chains, n_draws)
print("Divergent gaussian transitions:", np.sum(divergent))
print(divergent.shape)
# Plot trace for output[0,0] through output[4,0]

indices = [1, 3, 4, 12]
az.plot_trace(
    idata,
    var_names=["output"],
    coords={"output_dim_0": indices, "output_dim_1": [0]}
)
plt.savefig("figures_for_use_in_paper/trace_plots/Gauss.pdf", bbox_inches="tight")
plt.show()

In [ ]:
rhs_fit = tanh_fits['Friedman_N200_p10_sigma1.00_seed2']['Regularized Horseshoe tanh']['posterior']
idata = az.from_cmdstanpy(rhs_fit)
divergent = idata.sample_stats["diverging"].values  # shape (n_chains, n_draws)
print("Divergent RHS transitions:", np.sum(divergent))
print(divergent.shape)
indices = [1, 3, 4, 12]
az.plot_trace(
    idata,
    var_names=["output"],
    coords={"output_dim_0": indices, "output_dim_1": [0]}
)
plt.savefig("figures_for_use_in_paper/trace_plots/RHS.pdf", bbox_inches="tight")
plt.show()

In [ ]:
dhs_fit = tanh_fits['Friedman_N200_p10_sigma1.00_seed2']['Dirichlet Horseshoe tanh nodewise']['posterior']
idata = az.from_cmdstanpy(dhs_fit)
divergent = idata.sample_stats["diverging"].values  # shape (n_chains, n_draws)
print("Divergent DHS transitions:", np.sum(divergent))
print(divergent.shape)
indices = [1, 3, 4, 12]
az.plot_trace(
    idata,
    var_names=["output"],
    coords={"output_dim_0": indices, "output_dim_1": [0]}
)
plt.savefig("figures_for_use_in_paper/trace_plots/DHS.pdf", bbox_inches="tight")
plt.show()

In [ ]:
dhs_fit = tanh_fits['Friedman_N200_p10_sigma1.00_seed2']['Dirichlet Student T tanh nodewise']['posterior']
idata = az.from_cmdstanpy(dhs_fit)
divergent = idata.sample_stats["diverging"].values  # shape (n_chains, n_draws)
print("Divergent DST transitions:", np.sum(divergent))
print(divergent.shape)
indices = [1, 3, 4, 12]
az.plot_trace(
    idata,
    var_names=["output"],
    coords={"output_dim_0": indices, "output_dim_1": [0]}
)
plt.savefig("figures_for_use_in_paper/trace_plots/DST.pdf", bbox_inches="tight")
plt.show()

## RHAT PLOTS

In [23]:
import numpy as np
import matplotlib.pyplot as plt

abbr = {
    "Gaussian tanh": "Gauss",
    "Regularized Horseshoe tanh": "RHS",
    "Dirichlet Horseshoe tanh nodewise": "DHS",
    "Dirichlet Student T tanh nodewise": "DST",
}

def plot_output_rhats_tanh_overlay_by_N(
    rhats_by_N,          # dict: { "N=100": rhats_dict, "N=200": rhats_dict, ... }
    bins=60,
    log_y=True,
    figsize_per_plot=(3.6, 2.8),
    clip_q=0.999,
):
    """
    One row with one panel per model. In each panel, overlay histograms for each N (different colors).
    rhats_by_N[label] should be a dict {model_name -> Series_of_rhat}.
    """
    # Determine model order from the first entry
    first_key = next(iter(rhats_by_N))
    models = list(rhats_by_N[first_key].keys())
    ncols = len(models)

    fig, axes = plt.subplots(
        1, ncols,
        figsize=(figsize_per_plot[0] * ncols, figsize_per_plot[1]),
        sharey=True
    )
    if ncols == 1:
        axes = [axes]

    # Use a common x-range per model across N so overlays are comparable
    for ax, model in zip(axes, models):
        # collect all samples across N for that model to set a shared range
        all_x = []
        for _, rhats_dict in rhats_by_N.items():
            s = rhats_dict[model]
            x = np.asarray(s.values, dtype=float)
            x = x[np.isfinite(x)]
            if x.size:
                all_x.append(x)
        all_x = np.concatenate(all_x) if all_x else np.array([1.0])

        xmax = np.quantile(all_x, clip_q) if all_x.size else 1.1
        xrng = (1.0, xmax if xmax > 1.0 else 1.1)

        # overlay one histogram per N
        for label, rhats_dict in rhats_by_N.items():
            s = rhats_dict[model]
            x = np.asarray(s.values, dtype=float)
            x = x[np.isfinite(x)]
            ax.hist(
                x,
                bins=bins,
                range=xrng,
                histtype="step",   # outline so overlays remain readable
                linewidth=3,
                label=label,
                alpha=0.5
            )

        if log_y:
            ax.set_yscale("log")

        ax.set_xlabel(r"$\hat{R}$", fontsize=15)
        ax.set_title(abbr.get(model, model), fontsize=15)
        ax.tick_params(axis="both", labelsize=12)
        ax.grid(True, which="both", axis="y", alpha=0.3)

    axes[0].set_ylabel("Frequency", fontsize=15)

    # one shared legend for the whole figure
    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(handles, labels, loc="upper right", frameon=True)

    fig.tight_layout()
    plt.savefig("figures_for_use_in_paper/Friedman_rhat.pdf", bbox_inches="tight")
    plt.show()
    #return fig


In [ ]:
rhats_by_N = {
    "N=100": rhats_tanh["Friedman_N100_p10_sigma1.00_seed1"],
    "N=200": rhats_tanh["Friedman_N200_p10_sigma1.00_seed2"],
    "N=500": rhats_tanh["Friedman_N500_p10_sigma1.00_seed11"],
}

plot_output_rhats_tanh_overlay_by_N(rhats_by_N, bins=100, log_y=True)
# plt.savefig("figures_for_use_in_paper/abalone_sparsity_with_beta.pdf", bbox_inches="tight")
# plt.show()


In [ ]:
python3 utils/run_all_regression_models.py --model dirichlet_horseshoe_tanh_nodewise --output_dir results/regression/single_layer/tanh/friedman/convergence_extensive --warmup 5000 --sample 2000 --N 200

## TEST OF BETTER CONVERGENCE MODELS

In [14]:
from sklearn.model_selection import train_test_split
from utils.generate_data import sample_gaussian_copula_uniform

def generate_Friedman_data_v2(N=100, D=10, sigma=1.0, test_size=0.2, seed=42, standardize_y=True, return_scale=True):
    np.random.seed(seed)
    X = np.random.uniform(0, 1, size=(N, D))
    x0, x1, x2, x3, x4 = X[:, 0], X[:, 1], X[:, 2], X[:, 3], X[:, 4]

    y_clean = (
        10 * np.sin(np.pi * x0 * x1) +
        20 * (x2 - 0.5) ** 2 +
        10 * x3 +
        5.0 * x4
    )
    y = y_clean + np.random.normal(0, sigma, size=N)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

    if not standardize_y:
        return (X_train, X_test, y_train, y_test) if not return_scale else (X_train, X_test, y_train, y_test, 0.0, 1.0)

    y_mean = y_train.mean()
    y_std = y_train.std() if y_train.std() > 0 else 1.0

    y_train_s = (y_train - y_mean) / y_std
    y_test_s = (y_test - y_mean) / y_std

    if return_scale:
        return X_train, X_test, y_train_s, y_test_s, y_mean, y_std
    return X_train, X_test, y_train_s, y_test_s

def generate_correlated_Friedman_data_v2(N=100, D=10, sigma=1.0, test_size=0.2, seed=42, standardize_y=True, return_scale=True):
    """
    Generate synthetic regression data for Bayesian neural network experiments.

    Parameters:
        N (int): Number of samples.
        D (int): Number of features.
        sigma (float): Noise level.
        test_size (float): Proportion for test split.
        seed (int): Random seed.
        standardize_y (bool): Whether to standardize the response variable.

    Returns:
        tuple: (X_train, X_test, y_train, y_test, y_mean, y_std) if standardize_y,
               else (X_train, X_test, y_train, y_test)
    """
    np.random.seed(seed)
    d = 10
    S_custom = np.eye(d)
    # Block 1 (vars 0..4): high Spearman, 0.7
    for i in range(0, 3):
        for j in range(i+1, 3):
            S_custom[i, j] = S_custom[j, i] = 0.8
    # Block 2 (vars 5..9): moderate Spearman, 0.4
    for i in range(5, 10):
        for j in range(i+1, 10):
            S_custom[i, j] = S_custom[j, i] = -0.5
    # Cross-block weaker, 0.15
    for i in range(0, 5):
        for j in range(5, 10):
            S_custom[i, j] = S_custom[j, i] = 0.15
    # A couple of bespoke pairs:
    S_custom[0, 9] = S_custom[9, 0] = 0.4
    S_custom[2, 7] = S_custom[7, 2] = 0.9  # very strong (will be projected if infeasible)
    S_custom[3, 4] = S_custom[4, 3] = -0.9  # very strong (will be projected if infeasible)
    S_custom[1, 6] = S_custom[6, 1] = -0.9  # very strong (will be projected if infeasible)

    U, _ = sample_gaussian_copula_uniform(n=10000, S=S_custom, random_state=123)
    #X = np.random.uniform(0, 1, size=(N, D))
    if N != U.shape[0]:
        idx = np.random.choice(U.shape[0], size=N, replace=False)
        X = U[idx, :]
    else:
        X = U

    x0, x1, x2, x3, x4 = X[:, 0], X[:, 1], X[:, 2], X[:, 3], X[:, 4]

    y_clean = (
        10 * np.sin(np.pi * x0 * x1) +
        20 * (x2 - 0.5) ** 2 +
        10 * x3 +
        5.0 * x4
    )

    y = y_clean + np.random.normal(0, sigma, size=N)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

    if not standardize_y:
        return (X_train, X_test, y_train, y_test) if not return_scale else (X_train, X_test, y_train, y_test, 0.0, 1.0)

    y_mean = y_train.mean()
    y_std = y_train.std() if y_train.std() > 0 else 1.0

    y_train_s = (y_train - y_mean) / y_std
    y_test_s = (y_test - y_mean) / y_std

    if return_scale:
        return X_train, X_test, y_train_s, y_test_s, y_mean, y_std
    return X_train, X_test, y_train_s, y_test_s

def make_large_eval_set(
    generator_fn,
    N_train,
    D,
    sigma,
    seed,
    n_eval=5000,
    standardize_y=True
):
    """
    Returns X_eval, y_eval (standardized if standardize_y=True), plus y_mean,y_std
    defined from the training split.
    """
    N_total = N_train + n_eval

    X_tr, X_te, y_tr, y_te, y_mean, y_std = generator_fn(
        N=N_total, D=D, sigma=sigma, test_size=n_eval / N_total, seed=seed,
        standardize_y=standardize_y, return_scale=True
    )
    # Now X_te has approx n_eval points (exact given test_size construction).
    return X_te, np.asarray(y_te).squeeze(), y_mean, y_std

In [15]:
def compute_rmse(
    seeds, all_seeds, models, all_fits, get_N_sigma, forward_pass,
    folder,
    n_eval=5000,
    D=10,
    standardize_y=True,
    # pass the correct generator functions
    gen_uncorr=None,
    gen_corr=None,
):
    """
    Evaluate on a large generated test set instead of the stored tiny X_test/y_test.
    Assumes model was trained on standardized y if standardize_y=True.
    """
    assert gen_corr is not None #and gen_uncorr is not None, "Pass both generator functions."

    posterior_means = []

    # choose generator based on folder name
    def choose_gen(folder):
        return gen_corr if "friedman_correlated" in folder else gen_uncorr

    for seed in seeds:
        N, sigma = get_N_sigma(seed, all_seeds)
        dataset_key = f'Friedman_N{N}_p10_sigma{sigma:.2f}_seed{seed}'

        # Build large eval set consistent with training split standardization
        gen_fn = choose_gen(folder)
        X_test, y_test, y_mean, y_std = make_large_eval_set(
            generator_fn=gen_fn,
            N_train=N,
            D=D,
            sigma=sigma,
            seed=seed,
            n_eval=n_eval,
            standardize_y=standardize_y
        )

        for model in models:
            try:
                fit = all_fits[dataset_key][model]['posterior']
                W1_samples = fit.stan_variable("W_1")           # (S, P, H)
                W2_samples = fit.stan_variable("W_L")           # (S, H, O)
                b1_samples = fit.stan_variable("hidden_bias")   # (S, O, H)
                b2_samples = fit.stan_variable("output_bias")   # (S, O)

            except KeyError:
                print(f"[SKIP] Model or posterior not found: {dataset_key} -> {model}")
                continue

            S = W1_samples.shape[0]
            y_hats = np.zeros((S, y_test.shape[0]))
            for i in range(S):
                W1 = W1_samples[i]
                W2 = W2_samples[i]
                y_hat = forward_pass(X_test, W1, b1_samples[i][0], W2, b2_samples[i]).squeeze()
                y_hats[i] = y_hat
                
            posterior_mean = y_hats.mean(axis=0)
            posterior_mean_rmse = np.sqrt(np.mean((posterior_mean - y_test)**2))
            
            out_pm = {
                'seed': seed,
                'N': N,
                'sigma': sigma,
                'model': model,
                'n_eval': y_test.shape[0],
                'posterior_mean_rmse': posterior_mean_rmse,
                'posterior_mean_rmse_orig': posterior_mean_rmse * y_std,  # back to original y scale
            }
            
            posterior_means.append(out_pm)
            
    return pd.DataFrame(posterior_means)

def get_N_sigma(seed, all_seeds):
    if seed in all_seeds[0]:
        N=100
    elif seed in all_seeds[1]:
        N=200
    else:
        N=500
    sigma=1.00
    return N, sigma

In [16]:
from utils.sparsity import forward_pass_tanh
N100_seeds = [1, 3, 4, 5, 6]
N200_seeds = [2, 7, 8, 9, 10]
N500_seeds = [11, 12, 13, 14, 15]
all_seeds = [N100_seeds, N200_seeds, N500_seeds]

In [20]:
rmse_df = compute_rmse(
        seeds=[2, 7, 8, 9, 10],
        all_seeds=all_seeds,
        models=model_names_tanh,
        all_fits=tanh_fits,
        get_N_sigma=get_N_sigma,
        forward_pass=forward_pass_tanh,
        folder="friedman",
        n_eval=1000,
        D=10,
        standardize_y=True,
        gen_uncorr=generate_Friedman_data_v2,
        gen_corr=generate_correlated_Friedman_data_v2,
    )